# Exemple de notebook - filtres sql et récupération des données en pandas

L'objectif de ce notebook est de fournir des exemples pour pré-filtrer les données via sql avant de charger les données dans un DataFrame pandas. 


In [1]:
import pandas as pd

pd.set_option("display.max_columns", None)  # show all cols
pd.set_option("display.max_colwidth", None)  # show full width of showing cols
pd.set_option(
    "display.expand_frame_repr", False
)  # print cols side by side as it's supposed to be

In [2]:
# Nous commencons par importer les librairies nécessaires pour l'analyse des données.

import duckdb
from pipelines.tasks._common import DUCKDB_FILE

con = duckdb.connect(database=DUCKDB_FILE, read_only=True)

## Filtres
1. Filtrer sur les prélévements de 2024

In [ ]:
query_2024 = """
SELECT * from edc_prelevements
WHERE dateprel >= '2024-01-01'
"""

prelevements_2024 = con.sql(query_2024)
prelevements_2024_df = prelevements_2024.df()
prelevements_2024_df.head(2)

2. Filtrer sur les prélévements non conformes en 2024


In [ ]:
where_clause = """
"dateprel" >= '2024-01-01'
  AND (
    (
      "plvconformitebacterio" = 'N'
    )
    OR (
      "plvconformitechimique" = 'N'
    )
    OR (
      "plvconformitereferencebact" = 'N'
    )
    OR (
      "plvconformitereferencechim" = 'N'
    )
  )
"""
query_non_conforme = f"""
SELECT
  *
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
prelevements_2024_non_conforme = con.sql(query_non_conforme)
prelevements_2024_non_conforme_df = prelevements_2024_non_conforme.df()
prelevements_2024_non_conforme_df.head(2)

## Selectionner des colonnes avant d'exécuter la requête

Selectionner les colonnes avant de charger les données permets une exécution plus rapide et limite l'usage de la mémoire.

In [ ]:
query_preselected = f"""
SELECT
  "referenceprel",
  "dateprel",
  "nomcommuneprinc",
  "plvconformitebacterio"
FROM "edc_prelevements"
WHERE
  {where_clause}
"""
preselected = con.sql(query_preselected)
preselected_df = preselected.df()
preselected_df.head(2)

## Jointure
Joindre edc_prelevements et edc_resultats sur referenceprel pour obtenir les résultats associés à chaque prélèvement :

In [ ]:
query = f"""
SELECT
  "edc_prelevements"."referenceprel",
  "edc_prelevements"."dateprel",
  "edc_prelevements"."nomcommuneprinc",
  "edc_resultats"."libmajparametre",
  "edc_resultats"."insituana",
  "edc_resultats"."rqana",
  "edc_resultats"."cdunitereferencesiseeaux"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) AS edc_prelevements
INNER JOIN "edc_resultats"
  ON "edc_prelevements"."referenceprel" = "edc_resultats"."referenceprel"
"""


joined = con.sql(query)
joined_df = joined.df()
joined_df

## Groupby et aggregats
Nombre total de prélèvements non conforme par commune en 2024

In [ ]:
query = f"""
SELECT
  "nomcommuneprinc",
  COUNT("referenceprel") AS "nb_prelevements_non_conformes"
FROM (
  SELECT
    *
  FROM "edc_prelevements" 
  WHERE
    {where_clause}
) 
GROUP BY
  1
"""
grouped = con.sql(query)
grouped_df = grouped.df()
grouped_df.sort_values("nb_prelevements_non_conformes", ascending=False)

## Autres exemples :

In [ ]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb
# Faisons une requête SQL en utilisant duckdb via la librarie python pour lister les substances qui ont été recherchées
# et les trier par ordre décroissant de leur nombre d'occurrences

con.sql("""
    SELECT libmajparametre, COUNT(*) as count
    FROM edc_resultats
    GROUP BY libmajparametre
    ORDER BY count DESC
""").show()

In [ ]:
# Exemple issu du notebook premier notebook d'exemple : exemple.ipynb

# Enfin, terminons par lister les prélèvements effectués dans une commune donnée

nomcommune = "TOULOUSE"

con.sql(f"""
    SELECT *
    FROM edc_prelevements
    WHERE nomcommuneprinc = '{nomcommune}'
""").show()